In [1]:
import neuroglancer as ng
from neuroglancer.server import global_server_args
from ngtracts.tracts import TractSource
import numpy as np


In [2]:
# Orientation color coding following  Pajevic & Pierpaoli, MRM (1999)
# We follow the mirror + rotational symmetry convention
colormap_orient = """
vec3 colormapOrient(vec3 orient) {
  vec3 result;
  result.r = abs(orient[0]);
  result.g = abs(orient[1]);
  result.b = abs(orient[2]);
  return clamp(result, 0.0, 1.0);
}
"""

# Runing this in the skeleton shader uses orientation to color tracts
orient_shader = colormap_orient + """
#uicontrol bool orient_color checkbox(default=true)
void main() {
  if (orient_color)
    emitRGB(colormapOrient(orientation));
  else
  	emitDefault();
}
"""

mri_shader = """
#uicontrol float brightness slider(min=0.0, max=1.0, default=0.5)
void main() {
    float x = clamp(toNormalized(getDataValue()) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.r = -0.14861 * cosangle + 1.78277 * sinangle;
    result.g = -0.29227 * cosangle + -0.90649 * sinangle;
    result.b = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""

hip_shader = """
#uicontrol float brightness slider(min=0.0, max=1.0, default=0.5)
void main() {
    float x = clamp(((toNormalized(getDataValue()) - 22000.)/3000.) * brightness, 0.0, 1.0);
    float angle = 2.0 * 3.1415926 * (4.0 / 3.0 + x);
    float amp = x * (1.0 - x) / 2.0;
    vec3 result;
    float cosangle = cos(angle);
    float sinangle = sin(angle);
    result.r = -0.14861 * cosangle + 1.78277 * sinangle;
    result.g = -0.29227 * cosangle + -0.90649 * sinangle;
    result.b = 1.97294 * cosangle;
    result = clamp(x + amp * result, 0.0, 1.0);
    emitRGB(result);
}
"""


In [3]:
global_server_args['bind_port'] = '9999'
viewer = ng.Viewer(token='1')
print(viewer.get_viewer_url())


http://127.0.0.1:9999/v/1/


In [4]:
# figure out HiP-CT // MRI transform

vx=15.13E-3

# I made a mistake when I built the vox2ras of the lowres hipct
# This transforms maps from the true hip-ct RAS space to the "wrong" one
rasfix2rasnofix = (
    [[1.     ,  0.     ,  0.     ,  -.37896],
    [ 0.     ,  1.     ,  0.     ,  -.37896],
    [ 0.     ,  0.     ,  1.     ,  -.37896],
    [ 0.     ,  0.     ,  0.     ,  1.     ]]
)

# HIP2MRI ras2ras (copy-pasted from the LTA)
ras2ras = np.asarray([
    [7.898524999618530e-01, -6.120176240801811e-02, -7.702938914299011e-01, 1.671681823730469e+02],
    [2.419907413423061e-02, 1.071445226669312e+00, -3.254489228129387e-02, -1.881584472656250e+02],
    [8.082613945007324e-01, 8.847340941429138e-02, 7.198479175567627e-01, 9.310166931152344e+01],
    [0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00, 1.000000000000000e+00],
])

# HIP2HIP: reorient
pli2ras = np.asarray([
    [ 0.     ,  0.     , -1.     ,  0.     ],
    [-1.     ,  0.     ,  0.     ,  0.     ],
    [ 0.     , -1.     ,  0.     ,  0.     ],
    [ 0.     ,  0.     ,  0.     ,  1.     ],
])

phys2mri = ras2ras @ rasfix2rasnofix @ pli2ras


In [5]:
TRK = "https://dandiarchive.s3.amazonaws.com/blobs/d4a/c43/d4ac43bd-6896-4adf-a911-82edbea21f67"
NII = "https://dandiarchive.s3.amazonaws.com/blobs/3de/a2d/3dea2d82-8af8-434f-b7a9-60a21d891985"
LTA = "https://dandiarchive.s3.amazonaws.com/blobs/4a1/023/4a102340-906b-4ebb-bcf2-43b4655ad549"
HIP = "https://dandiarchive.s3.amazonaws.com/zarr/5c37c233-222f-4e60-96e7-a7536e08ef61/"

hip_transform = ng.CoordinateSpaceTransform(
    matrix=phys2mri[:3, :4],
    input_dimensions=ng.CoordinateSpace(
        names=["z", "y", "x"],
        units="mm",
        scales=[15.13e-3]*3,
    ),
    output_dimensions=ng.CoordinateSpace(
        names=["x", "y", "z"],
        units="mm",
        scales=[1]*3,
    )
)

with viewer.txn() as state:
    state.layers.append(
        name="mri",
        layer=ng.ImageLayer(
            source=["nifti://" + NII],
            shader=mri_shader,
        )
    )
    state.layers.append(
        name="hip-ct",
        layer=ng.ImageLayer(
            source=ng.LayerDataSource(
              url="zarr://" + HIP,
              transform=hip_transform
            ),
            # shader=hip_shader,
        )
    )
    state.layers.append(
        name="tracts",
        layer=ng.SegmentationLayer(
            source=[TractSource(TRK)],
            skeleton_shader=orient_shader,
            selected_alpha=0,
            not_selected_alpha=0,
            segments=[1],
        ),
    )
